## Imports

In [ ]:
import tensorflow as tf
import keras 
from keras import layers
from keras import optimizers
from keras import applications
from keras import losses
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2 as cv
import csv
from sklearn.utils import shuffle

print("Modules imported")

## Colab

<a href="https://colab.research.google.com/github/SethCram/CS474-Deep-Learning/blob/main/HW4/HW4_CNN_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Google Colab Cell

#enable debugging
!pip install -Uqq ipdb
import ipdb
%pdb on

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/School/Senior Year/CS 474-01 (Deep Learning)/

# Data Import

In [ ]:
#LOAD CSV AND IMAGES (in proper resolution)

df=pd.read_csv('https://raw.githubusercontent.com/SethCram/CS474-Deep-Learning/main/HW4/train.csv', sep = ',')
print(df)

training_imgs_path = 'trainImgs/'
test_imgs_path = 'testImgs/'

targetWidth = 512
targetHeight = 512

# store training and test imgs greyscaled 
x_train = np.array([
            cv.resize( #resize to desired size
                cv.imread(training_imgs_path + row[1] + ".png"), 
                dsize=(targetWidth, targetHeight),
            ) 
            for row in df.values
    ])
x_test = np.array([
            cv.resize( #resize to desired size
                cv.imread(test_imgs_path + str(i) + ".png"),
                dsize=(targetWidth, targetHeight),
            )
            for i in range(1248)
    ])

#store whether benign (0?) or malignant (1?)
y_train = np.array([row[2] for row in df.values], dtype='float32')

nImg = 4  
for i in range(nImg*nImg):
    plt.subplot(nImg, nImg, i+1)
    plt.imshow(x_train[i], cmap = 'Greys_r')
plt.show()

# Transfer Learning
## Model Creation

### VGG Model

In [ ]:
## VGG

#avg = 530
#avg_training_width = sum( image.shape[0] for image in x_train ) / x_train.shape[0]
#avg_training_height = sum( image.shape[1] for image in x_train ) / x_train.shape[0]
#avg_test_width = sum( image.shape[0] for image in x_test ) / x_test.shape[0]
#avg_test_height = sum( image.shape[1] for image in x_train ) / x_train.shape[0]

# layer construction
input_layer = layers.Input(shape=(targetWidth, targetHeight, 3))
preprocc_input = applications.vgg16.preprocess_input
base_model = applications.VGG16(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
flatten_layer = layers.Flatten()
fc_layer = layers.Dense(512, activation='relu')
# Add a dropout rate of 0.5
#dropout_layer = layers.Dropout(0.5)
output_layer = layers.Dense(
    units=1,
    activation='sigmoid'
)
base_model.trainable = False

#layer connecting
x = preprocc_input(input_layer)
x = base_model(x, training=False)
x = flatten_layer(x)
x = fc_layer(x)
#x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### InceptionV3 Model

In [ ]:
# layer construction
input_layer = layers.Input(shape=(targetWidth, targetHeight, 3))
preprocc_input = applications.inception_v3.preprocess_input

base_model = applications.InceptionV3(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
base_model.trainable = False

global_avg_pool_layer = layers.GlobalAveragePooling2D()
fc_layer = layers.Dense(1024, activation='relu')
#dropout_layer = layers.Dropout(0.2)
output_layer = layers.Dense(
    units=1,
    activation=keras.activations.sigmoid #'softmax'
)

#layer connecting
x = preprocc_input(input_layer)
x = base_model(x, training=False)
x = global_avg_pool_layer(x)
x = fc_layer(x)
#x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

### Efficient Net V2M

In [ ]:
## DOESN'T WORK

# layer construction
input_layer = layers.Input(shape=(targetWidth, targetHeight, 3))
preprocc_input = applications.efficientnet_v2.preprocess_input

base_model = applications.EfficientNetV2B0( #V2M(
    input_shape=(targetWidth, targetHeight, 3),
    include_top=False,
)
base_model.trainable = False

fc_layer = layers.Dense(1024, activation='relu')
dropout_layer = layers.Dropout(0.5) #0.5
output_layer = layers.Dense(
    units=1,
    activation=keras.activations.sigmoid 
)

#layer connecting
x = preprocc_input(input_layer)
x = base_model(x, training=False)
x = fc_layer(x)
x = dropout_layer(x)
predictions = output_layer(x)
model = keras.Model(input_layer, predictions)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))

## Model Compilation

In [ ]:

model.compile(
    optimizer=optimizers.RMSprop(), #'adam', #to deal with noise
    loss=losses.BinaryCrossentropy(), #or just categorical_crossentropy?
    metrics=['accuracy']
)

#model.summary(expand_nested=True, show_trainable=True)
model.summary()

## Restore Model

In [ ]:
## RESTORE THE MODEL

model = keras.models.load_model("saved_models/inceptronv3_dropout2_model")

model.summary(expand_nested=True, show_trainable=True)

### Fine Tuning

In [ ]:

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:

#base_model = model.layers[3]

model.layers[3].trainable = True

model.layers[3].layers[299].trainable = True

for layer in model.layers[3].layers[:299]:
   layer.trainable = False
for layer in model.layers[3].layers[300:]:
   layer.trainable = False
   
model.layers[3].layers[294].trainable = True

model.summary(expand_nested=True, show_trainable=True)

In [ ]:
#need to recompile model
model.compile(
    optimizer=optimizers.SGD(learning_rate=0.0001, momentum=0.9), #not sure why use SGD w/ mom
    loss=losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

## Model Training

In [ ]:
#tf.config.run_functions_eagerly(True)

#shuffle data everytime train
x_train, y_train = shuffle(x_train, y_train)

batch_size = 32

history = model.fit(
    x_train, #took first 500 for VGG
    y_train, #took first 500 for VGG
    epochs=10, #5, 
    batch_size=batch_size,  #128, 64 crashed it; 40, 20 in both Epoch ETA is 10 mins didn't crash it (VGG)
    validation_split=0.2, 
    validation_batch_size=batch_size
)

#history of training and validation accuracy (only works if model trained and not preloaded)
plt.plot(history.history['accuracy'], label='training acc')
plt.plot(history.history['val_accuracy'], label='validation acc')
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# Save Model

In [ ]:
## SAVE THE MODEL

model.save("saved_models/inceptronv3_dropout3_finetuned_model")

model.summary()

# Model Prediction

In [ ]:
## Model Prediction

batch_size = 32

#test prediction storage
y_test_pred = model.predict(x_test, batch_size=batch_size).flatten()

#round probabilities to 0 or 1
y_test_pred = np.round(y_test_pred)

with open('HW4_predictions_SethCram.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    
    for i in range(len(y_test_pred)):
        writer.writerow([str(i) + ".png", y_test_pred[i]])
        
#train accuracy
y_train_pred = model.predict(x_train, batch_size=batch_size).flatten()

#round probabilities to 0 or 1
y_train_pred = np.round(y_train_pred)

train_acc = np.sum(y_train == y_train_pred) / y_train.shape[0]

print(f'train accuracy: {train_acc * 100}%')

#apply preprocessing to x_test too
x_test = preprocc_input(x_test)
        